In [22]:
import pickle
import pandas as pd
import json

In [29]:
result_dir = 'result'
source_dir = 'source'

In [12]:
results_name = 'results_full_add_2024-10-24.pkl'

with open(f'{result_dir}/{results_name}', 'rb') as f:
    data = pickle.load(f)['original_response']

res = {row_n:res_dict['choices'][0]['message']['content'] for row_n, res_dict in data.items()}
res

{'0': '{"name": "billing_charge_analyze_summary", "arguments": {}}',
 '1': '{"name": "estimated_billing_charge_compare_to_current", "arguments": {"keywords": {"productName": "5GX플래티넘"}}}',
 '2': '{"name": "estimated_billing_charge_compare_to_current", "arguments": {"keywords": {"includedVoiceCall": ""}}}',
 '3': '{"name": "SEARCH_MOBILE_PLAN", "arguments": {"keywords": {"includedDataForSharingAndTethering": "10GB 이상", "monthlyPrice": "고액"}}}',
 '4': '{"name": "SEARCH_MOBILE_PLAN", "arguments": {"keywords": {"benefit": "FLO 할인 100%", "includedDataForSharingAndTethering": "10기가 이하"}}}',
 '5': '{"name": "COMPARE_CHANGE_MY_MOBILE_PLAN", "arguments": {"keywords": {"includedVoiceCallTospecifiedNumbers": "1시간 이상"}}}',
 '6': '{"name": "estimated_billing_charge_compare_to_current", "arguments": {"keywords": {"includedData": "100GB 이상"}}}',
 '7': '{"name": "SEARCH_MOBILE_PLAN", "arguments": {"keywords": {"includedVoiceCallTospecifiedNumbers": "60분", "onboardingChannel": "온라인", "includedVideoOrVa

In [39]:
df = pd.DataFrame.from_dict(res, orient='index', columns=['response'])
df['name'] = df['response'].apply(lambda x: json.loads(x)['name'])
df['arguments'] = df['response'].apply(lambda x: json.loads(x)['arguments'])
df.drop('response', axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)
df

,name,arguments
0,billing_charge_analyze_summary,{}
1,estimated_billing_charge_compare_to_current,{'keywords': {'productName': '5GX플래티넘'}}
2,estimated_billing_charge_compare_to_current,{'keywords': {'includedVoiceCall': ''}}
3,SEARCH_MOBILE_PLAN,{'keywords': {'includedDataForSharingAndTether...
4,SEARCH_MOBILE_PLAN,"{'keywords': {'benefit': 'FLO 할인 100%', 'inclu..."
5,COMPARE_CHANGE_MY_MOBILE_PLAN,{'keywords': {'includedVoiceCallTospecifiedNum...
6,estimated_billing_charge_compare_to_current,{'keywords': {'includedData': '100GB 이상'}}
7,SEARCH_MOBILE_PLAN,{'keywords': {'includedVoiceCallTospecifiedNum...
8,SEARCH_MOBILE_PLAN,"{'keywords': {'includedText': '무제한', 'included..."
9,check_plan_availability,{'requestForMultiline': False}


In [31]:
eval_file_name = "추가_평가_목록.xlsx"
eval_file_path = f'{source_dir}/{eval_file_name}' 
user_query_data = pd.read_excel(eval_file_path)

# Clean up column names by replacing spaces with underscores
user_query_data.columns = [c.replace(' ', '_') for c in user_query_data.columns]
user_query_data

,Feature_ID,Utterance_Sentence_ID,Utterance_Sentence,종합_테스트_결과,True_Intent
0,F01-I01,F01-I01_Temp_001,"이번달 좀 많이 쓴 것 같은데, 내 이번 달 청구금액이 얼만큼 더 추가될지 예상 가능해?",FAIL,billing_charge_analyze_summary
1,F06-I01,F06-I01_Temp_001,5GX플래티넘 요금제로 요금제 바꾸면 요금 얼마나 더 내야 돼?,FAIL,estimated_billing_charge_compare_to_current
2,F06-I01,F06-I01_Temp_002,음성통화 제공하는 요금제로 변경할때 예상요금 알려줘,FAIL,estimated_billing_charge_compare_to_current
3,F06-I01,F06-I01_Temp_030,테더링 10기가 이상 가능하고 가격이 꽤 나가는 요금제 가격 알려줘,FAIL,estimated_billing_charge_compare_to_current
4,F06-I01,F06-I01_Temp_031,FLO 할인이 100% 되는 혜택이랑 테더링 가능한 데이터가 10기가 이하인 요금제...,FAIL,estimated_billing_charge_compare_to_current
5,F06-I01,F06-I01_Temp_037,"엄마랑 1시간 이상 통화하는 요금제랑, 2시간 이상 통화하는 요금제로 변경하게 될 ...",FAIL,estimated_billing_charge_compare_to_current
6,F06-I01,F06-I01_Temp_038,"데이터가 기본적으로 50GB 이상 쓸 수 있는 요금제랑, 100GB 이상 쓸 수 있...",FAIL,estimated_billing_charge_compare_to_current
7,F06-I01,F06-I01_Temp_055,"할아버지와 60분의 무료 통화가 가능한 온라인 전용 요금제, 영상통화가 부족하지 않...",FAIL,estimated_billing_charge_compare_to_current
8,F06-I01,F06-I01_Temp_056,"문자메시지는 무제한 발신 가능, 한 사람과 과금 없이 5시간까지 통화 가능한 뉴 T...",FAIL,estimated_billing_charge_compare_to_current
9,F17-I08,F17-I08_Temp_004,다른 요금제 옵션으로 지금 변경 가능해?,FAIL,changable_date_for_plan


In [42]:
df_joined = pd.merge(user_query_data, df[['name']], how='inner', left_index=True, right_index=True)
df_joined.rename(columns={'name':'llm_output'}, inplace=True)
df_joined

,Feature_ID,Utterance_Sentence_ID,Utterance_Sentence,종합_테스트_결과,True_Intent,llm_output
0,F01-I01,F01-I01_Temp_001,"이번달 좀 많이 쓴 것 같은데, 내 이번 달 청구금액이 얼만큼 더 추가될지 예상 가능해?",FAIL,billing_charge_analyze_summary,billing_charge_analyze_summary
1,F06-I01,F06-I01_Temp_001,5GX플래티넘 요금제로 요금제 바꾸면 요금 얼마나 더 내야 돼?,FAIL,estimated_billing_charge_compare_to_current,estimated_billing_charge_compare_to_current
2,F06-I01,F06-I01_Temp_002,음성통화 제공하는 요금제로 변경할때 예상요금 알려줘,FAIL,estimated_billing_charge_compare_to_current,estimated_billing_charge_compare_to_current
3,F06-I01,F06-I01_Temp_030,테더링 10기가 이상 가능하고 가격이 꽤 나가는 요금제 가격 알려줘,FAIL,estimated_billing_charge_compare_to_current,SEARCH_MOBILE_PLAN
4,F06-I01,F06-I01_Temp_031,FLO 할인이 100% 되는 혜택이랑 테더링 가능한 데이터가 10기가 이하인 요금제...,FAIL,estimated_billing_charge_compare_to_current,SEARCH_MOBILE_PLAN
5,F06-I01,F06-I01_Temp_037,"엄마랑 1시간 이상 통화하는 요금제랑, 2시간 이상 통화하는 요금제로 변경하게 될 ...",FAIL,estimated_billing_charge_compare_to_current,COMPARE_CHANGE_MY_MOBILE_PLAN
6,F06-I01,F06-I01_Temp_038,"데이터가 기본적으로 50GB 이상 쓸 수 있는 요금제랑, 100GB 이상 쓸 수 있...",FAIL,estimated_billing_charge_compare_to_current,estimated_billing_charge_compare_to_current
7,F06-I01,F06-I01_Temp_055,"할아버지와 60분의 무료 통화가 가능한 온라인 전용 요금제, 영상통화가 부족하지 않...",FAIL,estimated_billing_charge_compare_to_current,SEARCH_MOBILE_PLAN
8,F06-I01,F06-I01_Temp_056,"문자메시지는 무제한 발신 가능, 한 사람과 과금 없이 5시간까지 통화 가능한 뉴 T...",FAIL,estimated_billing_charge_compare_to_current,SEARCH_MOBILE_PLAN
9,F17-I08,F17-I08_Temp_004,다른 요금제 옵션으로 지금 변경 가능해?,FAIL,changable_date_for_plan,check_plan_availability
